# Black 모형

선물(Futures) 혹은 선도(Forwards)를 기초자산으로 한 유러피언 옵션 프라이싱 목적

배당률, 보유비용, 편의수익률 직접 추정할 필요 X (선물을 기초자산으로 하므로)

## 코스피 200 옵션 프라이싱

3,6,9,12 월 옵션은 각각 만기에 맞는 선물 가격을 그대로 가져다 써도됨

In [2]:
import QuantLib as ql

In [3]:
valuationDate = ql.Date(25,4,2021)
ql.Settings.instance().evaluationDate = valuationDate
calendar = ql.SouthKorea()
dayCount = ql.ActualActual()

Simple Quote Objects

In [4]:
futures_qt = ql.SimpleQuote(426.90)    # KOSPI 200 Futures Apr 26
riskfreerate_qt = ql.SimpleQuote(0.01) # Risk=Free Rate
volatility_qt = ql.SimpleQuote(0.40)   # Volatility

Quote Handle Objects

In [5]:
f_qhd = ql.QuoteHandle(futures_qt)
r_qhd = ql.QuoteHandle(riskfreerate_qt)
v_qhd = ql.QuoteHandle(volatility_qt)

Term-Structure Objects

In [6]:
r_ts = ql.FlatForward(valuationDate, r_qhd, dayCount)
v_ts = ql.BlackConstantVol(valuationDate, calendar, v_qhd, dayCount)

Term-Structure Handle Objects

In [7]:
r_thd = ql.YieldTermStructureHandle(r_ts)
v_thd = ql.BlackVolTermStructureHandle(v_ts)

Process & Engine

In [8]:
process = ql.BlackProcess(f_qhd, r_thd, v_thd)   # 블랙숄즈 모형과 차이점
engine = ql.AnalyticEuropeanEngine(process)

Option Objects

In [9]:
option_type = ql.Option.Call
strikePrice = 400                                       #행사가
expiryDate = ql.Date(31,12,2021)
exercise = ql.EuropeanExercise(expiryDate)
payoff = ql.PlainVanillaPayoff(option_type, strikePrice)
option = ql.VanillaOption(payoff, exercise)

Pricing

In [10]:
option.setPricingEngine(engine)

Price & Greeks Results

In [11]:
print("Option Premium =", round(option.NPV(), 2))
print("Option Delta =", round(option.delta(), 4))
print("Option Gamma =", round(option.gamma(), 4))
print("Option Theta =", round(option.thetaPerDay(), 4))
print("Option Vega =", round(option.vega()/ 100, 4))
print("Option Rho =", round(option.rho()/ 100, 4))

Option Premium = 68.37
Option Delta = 0.637
Option Gamma = 0.0026
Option Theta = -0.103
Option Vega = 1.311
Option Rho = 1.3943


In [13]:
mkt_price = 70 # 콜옵션 가격
implied_volatility = option.impliedVolatility(mkt_price, process)
volatility_qt.setValue(implied_volatility) # 옵션의 시장가격과 확률과정을 받음
print("Option Premium =", round(option.NPV(), 2))
print("Option Delta =", round(option.delta(), 4))
print("Option Gamma =", round(option.gamma(), 4))
print("Option Theta =", round(option.thetaPerDay(), 4))
print("Option Vega =", round(option.vega()/ 100, 4))
print("Option Rho =", round(option.rho()/ 100, 4))

Option Premium = 70.0
Option Delta = 0.6367
Option Gamma = 0.0025
Option Theta = -0.1062
Option Vega = 1.3114
Option Rho = 1.3823
